Importing packages

In [3]:
import pandas as pd

Reading in the CSVs

In [4]:
full_moon = pd.read_csv("full_moon.csv")
holidays = pd.read_csv("holidays.csv")
weather = pd.read_csv("weather.csv")
crimes = pd.read_csv("Crimes.csv")

Recoding all the date columns to same format

In [5]:
## First, full_moon.csv

### Convert FullMoonDates column to datetime objects
full_moon['FullMoonDates'] = pd.to_datetime(full_moon['FullMoonDates'], format='%d %B %Y')

### Format the dates as 'YYYY-MM-DD' and assign them to a new column
full_moon['use_date'] = full_moon['FullMoonDates'].dt.strftime('%Y-%m-%d')

# Save the updated DataFrame back to the same CSV file, overwriting the original file
full_moon.to_csv("full_moon.csv", index=False)


## Next, holidays.csv

### Date column already in correct format, just want to change the name to use_date for consistency.
holidays.rename(columns={'Date': 'use_date'}, inplace=True)

# Save the updated DataFrame back to the same CSV file, overwriting the original file
holidays.to_csv("holidays.csv", index=False)


## Next, weather.csv

### Same situation as holidays.csv
weather.rename(columns={'datetime': 'use_date'}, inplace=True)

### Save the updated DataFrame back to the same CSV file, overwriting the original file
weather.to_csv("weather.csv", index=False)

ValueError: time data "2005-01-25" doesn't match format "%d %B %Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [6]:
## Now for Crimes.csv

### Convert the 'Date' column to datetime objects
crimes['Date'] = pd.to_datetime(crimes['Date'])

### Create new columns for 'use_date' and 'use_time'
crimes['use_date'] = crimes['Date'].dt.strftime('%Y-%m-%d')
crimes['use_time'] = crimes['Date'].dt.strftime('%H:%M:%S')

### Save the updated DataFrame back to the same CSV file, overwriting the original file
crimes.to_csv("Crimes.csv", index=False)

Inidicating if a day was a full moon or holiday

In [7]:
## Adding a full_moon binary value to weather.csv (since there's a row for each day in that csv)

# Convert 'use_date' column to datetime objects
weather['use_date'] = pd.to_datetime(weather['use_date'])
full_moon['use_date'] = pd.to_datetime(full_moon['use_date'])

# Create a new column 'full_moon' initialized with zeros
weather['full_moon'] = 0

# Set 'full_moon' column to 1 for dates in 'full_moon.csv'
weather.loc[weather['use_date'].isin(full_moon['use_date']), 'full_moon'] = 1

# Save the updated DataFrame back to the same CSV file, overwriting the original file
weather.to_csv("full1.csv", index=False)

In [8]:
## Adding a holiday binary value to weather.csv

# Convert 'use_date' column to datetime objects
weather['use_date'] = pd.to_datetime(weather['use_date'])
holidays['use_date'] = pd.to_datetime(holidays['use_date'])

# Create a new column 'holiday' initialized with zeros
weather['holiday'] = 0

# Set 'holiday' column to 1 for dates in 'holidays.csv'
weather.loc[weather['use_date'].isin(holidays['use_date']), 'holiday'] = 1

# Save the updated DataFrame back to the same CSV file, overwriting the original file
weather.to_csv("full1.csv", index=False)

Indicating if a crime happened, how many crimes, what types, etc.

In [9]:
# Convert 'use_date' column to datetime objects
full1 = pd.read_csv("full1.csv")
full1['use_date'] = pd.to_datetime(full1['use_date'])
crimes['use_date'] = pd.to_datetime(crimes['use_date'])

In [10]:
# Add binary column 'criminal_activity' to full1
full1['criminal_activity'] = 0
full1.loc[full1['use_date'].isin(crimes['use_date']), 'criminal_activity'] = 1

In [11]:
# Add 'crime_count' column to full1
crime_count_series = crimes.groupby('use_date').size()
crime_count_series.name = 'crime_count'  # Assign a name to the Series
full1_df = pd.merge(full1, crime_count_series, left_on='use_date', right_index=True, how='left')
full1_df['crime_count'].fillna(0, inplace=True)

In [12]:
# Add binary columns for each 'Primary Type' with count of occurrences on each day
crime_type_counts = crimes.groupby(['use_date', 'Primary Type']).size().unstack(fill_value=0)
crime_type_counts.reset_index(inplace=True)
full1_df = pd.merge(full1_df, crime_type_counts, on='use_date', how='left')

# Fill NaN values in crime type columns with 0
crime_type_columns = crime_type_counts.columns[1:]  # Exclude 'use_date' column
full1_df[crime_type_columns] = full1_df[crime_type_columns].fillna(0)

In [13]:
# Save the updated DataFrame back to the new CSV file
full1_df.to_csv("full2.csv", index=False)

Deciding Which Variables to Keep

In [17]:
data = full1_df[['use_date', "ASSAULT", "BATTERY", "CRIM SEXUAL ASSAULT", "CRIMINAL SEXUAL ASSAULT", "HUMAN TRAFFICKING", "KIDNAPPING", "HOMICIDE", 'crime_count', 'criminal_activity', 'holiday', 'full_moon', 'temp', 'feelslike', 'humidity', 'precip', 'precipcover', 'snow', 'windspeed', 'visibility', 'uvindex', 'sunrise', 'sunset', 'moonphase']]

In [18]:
data

,use_date,ASSAULT,BATTERY,CRIM SEXUAL ASSAULT,CRIMINAL SEXUAL ASSAULT,HUMAN TRAFFICKING,KIDNAPPING,HOMICIDE,crime_count,criminal_activity,...,humidity,precip,precipcover,snow,windspeed,visibility,uvindex,sunrise,sunset,moonphase
0,2010-01-01,2.0,10.0,8.0,0.0,0.0,0.0,0.0,74.0,1,...,69.8,0.000,0.00,0.0,21.2,15.1,3,2010-01-01T07:18:22,2010-01-01T16:30:09,0.53
1,2010-01-02,1.0,5.0,0.0,0.0,0.0,0.0,0.0,51.0,1,...,63.1,0.000,0.00,0.0,24.0,15.8,4,2010-01-02T07:18:26,2010-01-02T16:31:02,0.57
2,2010-01-03,6.0,7.0,0.0,0.0,0.0,0.0,0.0,44.0,1,...,66.0,0.000,0.00,0.0,24.2,15.6,4,2010-01-03T07:18:28,2010-01-03T16:31:56,0.60
3,2010-01-04,1.0,10.0,1.0,0.0,0.0,0.0,0.0,52.0,1,...,69.8,0.000,0.00,0.0,23.3,15.7,2,2010-01-04T07:18:27,2010-01-04T16:32:51,0.64
4,2010-01-05,2.0,6.0,0.0,0.0,0.0,0.0,0.0,52.0,1,...,70.8,0.000,0.00,0.0,21.9,15.4,3,2010-01-05T07:18:25,2010-01-05T16:33:49,0.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5168,2024-02-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,49.8,0.000,0.00,0.0,27.7,16.0,5,2024-02-25T06:32:21,2024-02-25T17:35:35,0.54
5169,2024-02-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,47.4,0.000,0.00,0.0,28.6,16.0,6,2024-02-26T06:30:49,2024-02-26T17:36:48,0.58
5170,2024-02-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,75.2,10.687,8.33,0.0,27.2,15.1,6,2024-02-27T06:29:16,2024-02-27T17:38:01,0.61
5171,2024-02-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,...,52.3,0.379,8.33,0.9,43.9,16.0,7,2024-02-28T06:27:42,2024-02-28T17:39:13,0.64
